# Import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Chargement des données

In [ ]:
train_data = pd.read_csv('../data/train.csv')
test_data = pd.read_csv('../data/test.csv')

In [ ]:
train_data.head(), test_data.head()

# Séparer features et target pour les données de train

In [ ]:
X_train = train_data.drop('label', axis=1).values
y_train = train_data['label'].values

# Normalisation des données

In [ ]:
X_train = X_train / 255.0 #Valeurs d'un pixel entre 0 et 1

In [ ]:
X_train = X_train.reshape(-1, 28, 28, 1) # Reshaper les données pour qu'elles conviennent au CNN (images de 28x28 pixels)

# Séparation des données train et val

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

# Préparation des données de test

In [ ]:
X_test = test_data.values / 255.0
X_test = X_test.reshape(-1, 28, 28, 1)

In [ ]:
X_train.shape, X_val.shape, X_test.shape

# Modèle CNN

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

# Compilation

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Early stopping

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Entrainement

In [ ]:
history = model.fit(X_train, y_train, epochs=20, validation_split=0.1, callbacks=[early_stopping])

In [ ]:
model.save('mnist_cnn_model.h5')

In [ ]:
plt.figure(figsize=(12, 5))

In [ ]:
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Loss d\'entraînement')
plt.plot(history.history['val_loss'], label='Loss de validation')
plt.title('Loss au cours de l\'entraînement')
plt.xlabel('Époque')
plt.ylabel('Loss')
plt.legend()

In [ ]:
history_dict = history.history

min_train_loss = min(history_dict['loss'])

print(min_train_loss)

In [ ]:
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Accuracy d\'entraînement')
plt.plot(history.history['val_accuracy'], label='Accuracy de validation')
plt.title('Accuracy au cours de l\'entraînement')
plt.xlabel('Époque')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
min_train_accuracy = max(history_dict['accuracy'])
print(min_train_accuracy)